# Analysis for Climate Discourse paper

This notebook contains the analysis for the 'substantive' paper

In [1]:
import pandas as pd
import statsmodels.formula.api as smd
import numpy as np

# Dataset overview and datawrangling

In [2]:
df = pd.read_csv('../data/annotations/twitter_klimaat_discussie_coderingen.csv')

In [3]:
df

,coder_id,jobset,scientific_consensus.pro_klimaat,scientific_consensus.belangrijk_onderwerp,scientific_consensus.sterke_mening,scientific_consensus.interesse_klimaat,age,gender,education,workhours,tweet_id,tweet_pro_klimaat,comment_nr,comment_pro_klimaat,comment_inhoudelijk_argument,comment_persoonlijke_aanval,comment_over_communicatie,comment_respectvol
0,11,set_3,3.0,4.0,5.0,5.0,30.0,Vrouw,Voorbereidend of kort middelbaar beroepsonderw...,Tussen de 10 en 20 uur,22,Ja,1,Ja,Inhoudelijk argument,Persoonlijke aanval,Oneens,Oneens
1,11,set_3,3.0,4.0,5.0,5.0,30.0,Vrouw,Voorbereidend of kort middelbaar beroepsonderw...,Tussen de 10 en 20 uur,22,Ja,2,Ja,Inhoudelijk argument,Persoonlijke aanval,Eens,Oneens
2,11,set_3,3.0,4.0,5.0,5.0,30.0,Vrouw,Voorbereidend of kort middelbaar beroepsonderw...,Tussen de 10 en 20 uur,22,Ja,3,Ja,Geen inhoudelijk argument,Geen persoonlijke aanval,Eens,Oneens
3,11,set_3,3.0,4.0,5.0,5.0,30.0,Vrouw,Voorbereidend of kort middelbaar beroepsonderw...,Tussen de 10 en 20 uur,22,Ja,4,Neutraal,Inhoudelijk argument,Persoonlijke aanval,Eens,Eens
4,11,set_3,3.0,4.0,5.0,5.0,30.0,Vrouw,Voorbereidend of kort middelbaar beroepsonderw...,Tussen de 10 en 20 uur,22,Ja,5,Neutraal,Inhoudelijk argument,Geen persoonlijke aanval,Oneens,Eens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43177,2588,set_44,6.0,7.0,6.0,6.0,22.0,Vrouw,"Hoger beroepsonderwijs (HBO, HTS, HEAO, Social...",Tussen de 30 en 40 uur,378,Ja,1,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43178,2588,set_44,6.0,7.0,6.0,6.0,22.0,Vrouw,"Hoger beroepsonderwijs (HBO, HTS, HEAO, Social...",Tussen de 30 en 40 uur,378,Ja,2,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43179,2588,set_44,6.0,7.0,6.0,6.0,22.0,Vrouw,"Hoger beroepsonderwijs (HBO, HTS, HEAO, Social...",Tussen de 30 en 40 uur,378,Ja,3,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43180,2588,set_44,6.0,7.0,6.0,6.0,22.0,Vrouw,"Hoger beroepsonderwijs (HBO, HTS, HEAO, Social...",Tussen de 30 en 40 uur,378,Ja,4,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens


In [4]:
# for this paper, we ignore the annotator characteritics for now - they need to be analyzed in the bias paper.
df2 = df[['coder_id', 'tweet_id', 'comment_nr',
   'comment_pro_klimaat',
    'comment_inhoudelijk_argument',
    'comment_persoonlijke_aanval',
    'comment_over_communicatie',
    'comment_respectvol']]
df2

,coder_id,tweet_id,comment_nr,comment_pro_klimaat,comment_inhoudelijk_argument,comment_persoonlijke_aanval,comment_over_communicatie,comment_respectvol
0,11,22,1,Ja,Inhoudelijk argument,Persoonlijke aanval,Oneens,Oneens
1,11,22,2,Ja,Inhoudelijk argument,Persoonlijke aanval,Eens,Oneens
2,11,22,3,Ja,Geen inhoudelijk argument,Geen persoonlijke aanval,Eens,Oneens
3,11,22,4,Neutraal,Inhoudelijk argument,Persoonlijke aanval,Eens,Eens
4,11,22,5,Neutraal,Inhoudelijk argument,Geen persoonlijke aanval,Oneens,Eens
...,...,...,...,...,...,...,...,...
43177,2588,378,1,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43178,2588,378,2,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43179,2588,378,3,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens
43180,2588,378,4,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Oneens


# TODO: SOME ERROR IN THE CSV
**As we see below, it seems that some values are shifted by a colum. These are very few, so proceed for now, but need to fix**

In [5]:
df2.comment_persoonlijke_aanval.value_counts()

comment_persoonlijke_aanval
Geen persoonlijke aanval     27075
Persoonlijke aanval          16046
Inhoudelijk argument            25
Geen inhoudelijk argument       18
Name: count, dtype: int64

In [6]:
pd.crosstab(df2.comment_persoonlijke_aanval, df2.comment_over_communicatie)

comment_over_communicatie,Eens,Geen persoonlijke aanval,Oneens,Persoonlijke aanval
comment_persoonlijke_aanval,,,,
Geen inhoudelijk argument,8,3,7,0
Geen persoonlijke aanval,12499,13,14546,8
Inhoudelijk argument,16,0,8,1
Persoonlijke aanval,10792,5,5236,12


## Flatten dataframe
We have multiple codings per comment - this will be analyzed in the bias paper.
For now, we proceed by taking the majority vote without further analysis

In [7]:
def aggcoders(series):
    '''takes mean if numeric, otherwise majority vote'''
    try:
        return series.mean()
    except:
        r = series.mode()
        return r
    
def tiebreak(cell):
    # we're conservative - if in doubt, we say not present. No > neutral > yes
    # superlelijke implementatie maar goed
    if pd.api.types.is_list_like(cell):
        for e in cell:
            if 'geen' or 'oneens' in e.lower():
                return e
        # if that didn't work, check for neutral
        for e in cell:
            if 'neutral' in e.lower():
                return e
        return
    else:
        return cell

In [8]:
# majority vote w/tiebreak
# TODO: maybe find better way once coder bias is analyzed
dfagg = df2.groupby(['tweet_id','comment_nr']).agg(aggcoders).reset_index().applymap(tiebreak)
dfagg

,tweet_id,comment_nr,coder_id,comment_pro_klimaat,comment_inhoudelijk_argument,comment_persoonlijke_aanval,comment_over_communicatie,comment_respectvol
0,1,1,1252.083333,Neutraal,Geen inhoudelijk argument,Geen persoonlijke aanval,Eens,Eens
1,1,2,1252.083333,Ja,Inhoudelijk argument,Geen persoonlijke aanval,Eens,Eens
2,1,3,1252.083333,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Eens,Eens
3,1,4,1252.083333,Ja,Inhoudelijk argument,Persoonlijke aanval,Eens,Eens
4,1,5,1252.083333,Nee,Geen inhoudelijk argument,Persoonlijke aanval,Eens,Oneens
...,...,...,...,...,...,...,...,...
1999,595,1,1182.272727,Nee,Geen inhoudelijk argument,Geen persoonlijke aanval,Oneens,Eens
2000,595,2,1182.272727,Neutraal,Geen inhoudelijk argument,Geen persoonlijke aanval,Eens,Oneens
2001,595,3,1182.272727,Nee,Geen inhoudelijk argument,Persoonlijke aanval,Eens,Oneens
2002,595,4,1182.272727,Nee,Geen inhoudelijk argument,Persoonlijke aanval,Eens,Oneens


In [9]:
# let's make a binary recode
def customrecode(x):
    if 'geen' in x.lower():
        return False
    elif 'oneens' in x.lower():
        return False
    else:
        return True
dfagg.iloc[:,4:] = dfagg.iloc[:,4:].applymap(customrecode)
dfagg

,comment_inhoudelijk_argument,comment_persoonlijke_aanval,comment_over_communicatie,comment_respectvol
0,False,False,True,True
1,True,False,True,True
2,False,False,True,True
3,True,True,True,True
4,False,True,True,False
...,...,...,...,...
1999,False,False,False,True
2000,False,False,True,False
2001,False,True,True,False
2002,False,True,True,False


# Create discussion-level varbiables


In [29]:
# is het hele gesprek alleen inhoudelijk en geen persoonlijke aanval?
gespreksniveau = pd.DataFrame(dfagg.groupby('tweet_id')['comment_inhoudelijk_argument'].min())
gespreksniveau.columns = ['helegesprekinhoudelijk']
gespreksniveau['helegesprekgeenpersoonlijkeaanval'] = dfagg.groupby('tweet_id')['comment_persoonlijke_aanval'].min()
gespreksniveau['issueframedgeprek'] = gespreksniveau['helegesprekgeenpersoonlijkeaanval'] * gespreksniveau['helegesprekinhoudelijk']
gespreksniveau['issueframedgeprek'] = gespreksniveau['issueframedgeprek'].map(bool)
gespreksniveau['helegesprekminimaaleendisrespect'] = dfagg.groupby('tweet_id')['comment_respectvol'].max()
gespreksniveau = gespreksniveau.reset_index()

gespreksniveau

,tweet_id,helegesprekinhoudelijk,helegesprekgeenpersoonlijkeaanval,issueframedgeprek,helegesprekminimaaleendisrespect
0,1,False,False,False,True
1,2,False,False,False,True
2,3,False,False,False,True
3,4,True,False,False,True
4,5,False,False,False,True
...,...,...,...,...,...
344,589,False,False,False,True
345,591,False,False,False,True
346,592,False,True,False,True
347,594,True,False,False,True


In [38]:
# How many discussions relevant to our hypotheses are there at all?
print("Percentages:\n")
print(gespreksniveau.drop('tweet_id', axis=1).applymap(int).mean())
print('\n\n')
print("Absolute numbers:\n")
print(gespreksniveau.drop('tweet_id', axis=1).applymap(int).sum())

Percentages:

helegesprekinhoudelijk               0.094556
helegesprekgeenpersoonlijkeaanval    0.028653
issueframedgeprek                    0.000000
helegesprekminimaaleendisrespect     0.985673
dtype: float64



Absolute numbers:

helegesprekinhoudelijk                33
helegesprekgeenpersoonlijkeaanval     10
issueframedgeprek                      0
helegesprekminimaaleendisrespect     344
dtype: int64


In [39]:
# SCRATCH BELOW

# H1 
H1: Interactions that include solely issue framing, polarize significantly less than interactions in which negative identity labels and/or process framing are present.

# TODO
waarschijnlijk is process framing verkeerd gecodeerd - zou bijna ooit Eens moeten zijn

In [17]:
gespreksniveau.issueframedgeprek.value_counts()

issueframedgeprek
0    199
Name: count, dtype: int64

In [18]:
gespreksniveau.helegesprekgeenpersoonlijkeaanval.value_counts()

helegesprekgeenpersoonlijkeaanval
False    196
True       3
Name: count, dtype: int64

In [19]:
gespreksniveau.helegesprekinhoudelijk.value_counts()

helegesprekinhoudelijk
False    316
True      33
Name: count, dtype: int64

In [20]:
gespreksniveau.helegesprekminimaaleendisrespect.value_counts()

helegesprekminimaaleendisrespect
True     196
False      3
Name: count, dtype: int64

In [21]:
#issue_frame_interacties = dfagg.query("comment_inhoudelijk_argument=='Inhoudelijk argument' & comment_persoonlijke_aanval=='Geen persoonlijke aanval'" )
#issue_frame_interacties

In [22]:
def gesprek_is_issue_only(series):
    return series.('Inhoudelijk argument')

SyntaxError: invalid syntax (3532696481.py, line 2)

In [ ]:
dfagg

In [ ]:
)

In [ ]:
inhoudelijkegesprekken = pd.DataFrame(test.groupby('tweet_id')['comment_inhoudelijk_argument_bool'].min())
# nee klopt niet, moet ook geen persoonlijke aanval
inhoudelijkegesprekken.columns=['inhoudelijkgesprek']
inhoudelijkegesprekken = inhoudelijkegesprekken.reset_index()

In [ ]:
test2 = test.merge(inhoudelijkegesprekken, on = 'tweet_id')
test2

In [ ]:
test2.inhoudelijkgesprek.value_counts()

In [ ]:
pd.crosstab(test2.inhoudelijkgesprek, test2.comment_respectvol)

In [ ]:
issue_frame_interacties.comment_respectvol.value_counts(normalize=True)

In [ ]:
rest = dfagg[~dfagg.index.isin(issue_frame_interacties.index)]
rest.comment_respectvol.value_counts(normalize=True)

In [ ]:
# nog heel veel te doen, mitsen maren ....
# maar: het is wel zou dat inhoudelijk + niet persoonlijk lijdt tot VEEL minder respectloze comments

# H2: Interactions that include negative identity labels, polarize significantly more often than interactions where these labels are not present.

In [ ]:
dfagg.groupby("comment_persoonlijke_aanval")['comment_respectvol']
